In [16]:
import pandas as pd
from scipy import stats
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
dataframes=[]
for i in range(1,31):
    df = pd.read_csv(f'/content/drive/MyDrive/Aras/House A/DAY_{i}.txt', sep=' ', header=None)
    dataframes.append(df)

In [19]:
columns = [
    "photocell_wardrobe",         # Column 1: Ph1 - Photocell in Wardrobe
    "photocell_couch",            # Column 2: Ph2 - Photocell in Convertible Couch (Resident 2's bed)
    "ir_tv_receiver",             # Column 3: Ir1 - IR Sensor at TV receiver
    "force_couch_1",              # Column 4: Fo1 - Force Sensor on Couch (1)
    "force_couch_2",              # Column 5: Fo2 - Force Sensor on Couch (2)
    "distance_chair_1",           # Column 6: Di3 - Distance Sensor at Chair (1)
    "distance_chair_2",           # Column 7: Di4 - Distance Sensor at Chair (2)
    "photocell_fridge",           # Column 8: Ph3 - Photocell at Fridge
    "photocell_kitchen_drawer",   # Column 9: Ph4 - Photocell at Kitchen Drawer
    "photocell_wardrobe_2",       # Column 10: Ph5 - Photocell in Wardrobe (another one)
    "photocell_bathroom_cabinet", # Column 11: Ph6 - Photocell at Bathroom Cabinet
    "contact_house_door",         # Column 12: Co1 - Contact Sensor at House Door
    "contact_bathroom_door",      # Column 13: Co2 - Contact Sensor at Bathroom Door
    "contact_shower_door",        # Column 14: Co3 - Contact Sensor at Shower Cabinet Door
    "sonar_hall",                 # Column 15: So1 - Sonar Distance Sensor in Hall
    "sonar_kitchen",              # Column 16: So2 - Sonar Distance Sensor in Kitchen
    "distance_tap",               # Column 17: Di1 - Distance Sensor at Tap
    "distance_water_closet",      # Column 18: Di2 - Distance Sensor at Water Closet
    "temperature_kitchen",        # Column 19: Te1 - Temperature Sensor in Kitchen
    "force_bed",                  # Column 20: Fo3 - Force Sensor on Bed
    "Resident1",                  # Column 21: Activity labels - Resident 1,
    "Resident2"                   # Column 22: Activity labels - Resident 2
]

In [20]:
activity_dict = {
    1: "Other",
    2: "Going Out",
    3: "Preparing Breakfast",
    4: "Having Breakfast",
    5: "Preparing Lunch",
    6: "Having Lunch",
    7: "Preparing Dinner",
    8: "Having Dinner",
    9: "Washing Dishes",
    10: "Having Snack",
    11: "Sleeping",
    12: "Watching TV",
    13: "Studying",
    14: "Having Shower",
    15: "Toileting",
    16: "Napping",
    17: "Using Internet",
    18: "Reading Book",
    19: "Laundry",
    20: "Shaving",
    21: "Brushing Teeth",
    22: "Talking on the Phone",
    23: "Listening to Music",
    24: "Cleaning",
    25: "Having Conversation",
    26: "Having Guest",
    27: "Changing Clothes"
}

Add hour column and chang column names

In [21]:
for df in dataframes:
    df.columns=columns
    df['Hour'] = df.index // 3600  # Assuming each row is one second, calculate the hour of day

Check if each day has 86,400 records (one record every second)

In [22]:
for df in dataframes:
    if(len(df)!=86400):
        print("Error", len(df))

Check if null values exist

In [23]:
flag=1
for df in dataframes:
    missing_values = df.isnull().sum()
    if(len(missing_values[missing_values >0])):
        print(missing_values[missing_values >0])
    else:
        flag=0
if flag==0:
    print('No missing values in the dataset')

No missing values in the dataset


In [24]:
dataframes[0].columns

Index(['photocell_wardrobe', 'photocell_couch', 'ir_tv_receiver',
       'force_couch_1', 'force_couch_2', 'distance_chair_1',
       'distance_chair_2', 'photocell_fridge', 'photocell_kitchen_drawer',
       'photocell_wardrobe_2', 'photocell_bathroom_cabinet',
       'contact_house_door', 'contact_bathroom_door', 'contact_shower_door',
       'sonar_hall', 'sonar_kitchen', 'distance_tap', 'distance_water_closet',
       'temperature_kitchen', 'force_bed', 'Resident1', 'Resident2', 'Hour'],
      dtype='object')

Check for invalid numbers

In [25]:
for df in dataframes:
    for i in range(0,20):
        for j in df[df.columns[i]]:
            if (j!=0 and j!=1):
                print("Invalid")
    for i in range(20,22):
        for j in df[df.columns[i]]:
            if (j<1 or j>27):
                print("Invalid")

In [26]:
for i in range(0,len(dataframes)):
    dataframes[i]['Day']=i+1

In [27]:
train=pd.DataFrame()
test=pd.DataFrame()
for df in dataframes[0:22]:
    train=pd.concat([train,df])
for df in dataframes[22:]:
    test=pd.concat([test,df])

In [28]:
print(train.shape)
print(test.shape)

(1900800, 24)
(691200, 24)


In [29]:
scaler = StandardScaler()
sensor_data_scaled = scaler.fit_transform(test.iloc[:40000, :20])

In [30]:
# Step 2: Dimensionality Reduction (optional but recommended)
pca = PCA(n_components=5)  # Reduce dimensions to 5 principal components
sensor_data_pca = pca.fit_transform(sensor_data_scaled)

In [31]:

# Step 3: Apply DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust eps and min_samples based on your data
dbscan_labels = dbscan.fit_predict(sensor_data_pca)

# Step 4: Identify anomalies
# Points labeled as -1 are considered anomalies by DBSCAN
anomalies = test.iloc[:40000, :20][dbscan_labels == -1]
print("Number of anomalies detected:", len(anomalies))

Number of anomalies detected: 64


In [32]:
anomalies

,photocell_wardrobe,photocell_couch,ir_tv_receiver,force_couch_1,force_couch_2,distance_chair_1,distance_chair_2,photocell_fridge,photocell_kitchen_drawer,photocell_wardrobe_2,photocell_bathroom_cabinet,contact_house_door,contact_bathroom_door,contact_shower_door,sonar_hall,sonar_kitchen,distance_tap,distance_water_closet,temperature_kitchen,force_bed
1468,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3411,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3473,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3490,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3496,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39633,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
39634,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
39635,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
39636,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


One Class SVM

In [42]:
# Step 1: Standardize the test data
scaler = StandardScaler()
sensor_data_scaled = scaler.fit_transform(test.iloc[:40000, :20])

# Step 2: Dimensionality Reduction using PCA
pca = PCA(n_components=5)  # Reduce dimensions to 5 principal components
sensor_data_pca = pca.fit_transform(sensor_data_scaled)

# Step 3: Train the One-Class SVM model
ocsvm = OneClassSVM(nu=0.1, kernel="rbf", gamma='scale')  # nu is the upper bound on the fraction of outliers
ocsvm.fit(sensor_data_pca)

# Step 4: Predict anomalies (outliers) on the test data
ocsvm_predictions = ocsvm.predict(sensor_data_pca)

# Print the first few predictions to see the raw output
print(ocsvm_predictions[:10])  # This will show you the raw output of -1 and 1

# Step 5: Identifying and count anomalies
# The model will return -1 for outliers and 1 for normal data
anomalies = test.iloc[:40000, :20][ocsvm_predictions == -1]
print("Number of anomalies detected:", len(anomalies))
print(anomalies.head())


[1 1 1 1 1 1 1 1 1 1]
Number of anomalies detected: 23220
    photocell_wardrobe  photocell_couch  ir_tv_receiver  force_couch_1  \
45                   0                0               0              0   
51                   0                0               0              0   
52                   0                0               0              0   
53                   0                0               0              0   
54                   0                0               0              0   

    force_couch_2  distance_chair_1  distance_chair_2  photocell_fridge  \
45              1                 0                 0                 0   
51              1                 0                 0                 0   
52              1                 0                 0                 0   
53              1                 0                 0                 0   
54              1                 0                 0                 0   

    photocell_kitchen_drawer  photocell_wardro